In [1]:
!pip install gdown==5.2.0
!pip install matplotlib==3.7.2
!pip install numpy==1.24.3
!pip install pandas==2.0.3
!pip install scikit-learn==1.3.0
!pip install torch==2.1.2
!pip install torch-geometric==2.5.3

  Using cached scikit_learn-1.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.1

In [2]:
"""
Dataset download from Google Drive using gdown
"""

!git clone https://github.com/hyein99/CS471_Pokemon_battle_prediction.git


Cloning into 'CS471_Pokemon_battle_prediction'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 41 (delta 14), reused 25 (delta 5), pack-reused 0
Receiving objects: 100% (41/41), 308.60 KiB | 1.21 MiB/s, done.
Resolving deltas: 100% (14/14), done.


# Import Modules

In [3]:
import random
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
import copy

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [4]:
SEED = 42
deterministic = True

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn

# Data Pre-processing

In [5]:
# Load pokemon.csv and combats.csv
pokemon_data = pd.read_csv('./CS471_Pokemon_battle_prediction/dataset/pokemon.csv')
combats_data = pd.read_csv('./CS471_Pokemon_battle_prediction/dataset/combats.csv')

# Encode vertex values to unique integers
label_encoder = LabelEncoder()
pokemon_data['#'] = label_encoder.fit_transform(pokemon_data['#'])
combats_data[['First_pokemon', 'Second_pokemon', 'Winner']] = \
    combats_data[['First_pokemon', 'Second_pokemon', 'Winner']].apply(label_encoder.transform)

features_to_normalize = ['HP', 'Attack', 'Defense', 'Speed', 'Generation', 'Sp. Atk', 'Sp. Def']
features_else = ['Type 1_Bug', 'Type 1_Dark', 'Type 1_Dragon', 'Type 1_Electric',
              'Type 1_Fairy', 'Type 1_Fighting', 'Type 1_Fire', 'Type 1_Flying',
              'Type 1_Ghost', 'Type 1_Grass', 'Type 1_Ground', 'Type 1_Ice',
              'Type 1_Normal', 'Type 1_Poison', 'Type 1_Psychic', 'Type 1_Rock',
              'Type 1_Steel', 'Type 1_Water', 'Type 2_Bug', 'Type 2_Dark',
              'Type 2_Dragon', 'Type 2_Electric', 'Type 2_Fairy', 'Type 2_Fighting',
              'Type 2_Fire', 'Type 2_Flying', 'Type 2_Ghost', 'Type 2_Grass',
              'Type 2_Ground', 'Type 2_Ice', 'Type 2_Normal', 'Type 2_Poison',
              'Type 2_Psychic', 'Type 2_Rock', 'Type 2_Steel', 'Type 2_Water']

scaler = MinMaxScaler()

pokemon_data[features_to_normalize] = \
    scaler.fit_transform(pokemon_data[features_to_normalize])


pokemon_data = pd.get_dummies(pokemon_data, columns=['Type 1', 'Type 2'])
pokemon_data[features_else] = \
    pokemon_data[features_else].astype(int)

hp_weight = 5.0
pokemon_data['HP'] *= hp_weight

pokemon_data['Legendary'] = pokemon_data['Legendary'].astype(int)
pokemon_data['total_stat'] = pokemon_data[features_to_normalize].sum(axis=1)

In [6]:
# Split combats data into train and test
train_combats, test_combats = train_test_split(combats_data, test_size=0.2, random_state=SEED)

# Extract unique vertex values from train_combats and test_combats
train_vertices = set(train_combats['First_pokemon']).union(set(train_combats['Second_pokemon']))
test_vertices = set(test_combats['First_pokemon']).union(set(test_combats['Second_pokemon']))

# Split pokemon data into train and test based on the vertices
train_pokemon = pokemon_data[pokemon_data['#'].isin(train_vertices)]
test_pokemon = pokemon_data[pokemon_data['#'].isin(test_vertices)]

# Decode vertex values back to original values if needed
train_pokemon['#'] = label_encoder.inverse_transform(train_pokemon['#'])
test_pokemon['#'] = label_encoder.inverse_transform(test_pokemon['#'])

# Set "#" as index for train_pokemon and test_pokemon dataframes
train_pokemon.set_index('#', inplace=True)
test_pokemon.set_index('#', inplace=True)

<ipython-input-6-48927d68a4a0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pokemon['#'] = label_encoder.inverse_transform(train_pokemon['#'])
<ipython-input-6-48927d68a4a0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pokemon['#'] = label_encoder.inverse_transform(test_pokemon['#'])


In [7]:
features = ['total_stat']+features_to_normalize + features_else

X_data = pokemon_data[features].values
X_data = torch.tensor(X_data, dtype=torch.float)

# for train dataset
X_train = train_pokemon[features].values
X_train = torch.tensor(X_train, dtype=torch.float)
edges_train = []
neg_edge_index_train = []

for _, row in train_combats.iterrows():
    first_pokemon = row['First_pokemon']
    second_pokemon = row['Second_pokemon']
    winner = row['Winner']

    if first_pokemon == winner:
      edges_train.append((second_pokemon, first_pokemon))
      neg_edge_index_train.append((first_pokemon, second_pokemon))

    else:
      edges_train.append((first_pokemon, second_pokemon))
      neg_edge_index_train.append((second_pokemon, first_pokemon))

neg_edge_index_train = torch.tensor(neg_edge_index_train, dtype=torch.long).t()
edge_index_train = torch.tensor(edges_train, dtype=torch.long).t()

data_train = Data(x=X_data, edge_index=edge_index_train, neg_edge_index = neg_edge_index_train)
train_loader = DataLoader([data_train], batch_size=1, shuffle=True)

# for test dataset
X_test = test_pokemon[features].values
X_test = torch.tensor(X_test, dtype=torch.float)
edges_test = []
neg_edge_index_test = []

for _, row in test_combats.iterrows():
    first_pokemon = row['First_pokemon']
    second_pokemon = row['Second_pokemon']
    winner = row['Winner']

    if first_pokemon == winner:
      edges_test.append((second_pokemon, first_pokemon))
      neg_edge_index_test.append((first_pokemon, second_pokemon))

    else:
      edges_test.append((first_pokemon, second_pokemon))
      neg_edge_index_test.append((second_pokemon, first_pokemon))

edge_index_test = torch.tensor(edges_test, dtype=torch.long).t()
neg_edge_index_test = torch.tensor(neg_edge_index_test, dtype=torch.long).t()

data_test = Data(x=X_data, edge_index=edge_index_test, neg_edge_index= neg_edge_index_test)
test_loader = DataLoader([data_test], batch_size=1, shuffle=False)

# for total data
edge = edges_train
edge.extend(edges_test)
edge_index = torch.tensor(edge, dtype=torch.long).t()

data_total = Data(x=X_data, edge_index = edge_index)

# Model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# GCN and GraphSage
class GraphSageLayer(nn.Module):
    def __init__(self, dim_in: int, dim_out: int, agg_type: str):
        super(GraphSageLayer, self).__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.agg_type = agg_type
        self.act = nn.ReLU()

        if self.agg_type == 'gcn':
            self.weight = nn.Linear(self.dim_in, self.dim_out, bias=False)
            self.bias = nn.Linear(self.dim_in, self.dim_out, bias=False)

        elif self.agg_type == 'mean':
            self.weight = nn.Linear(2 * self.dim_in, self.dim_out, bias=False)

        elif self.agg_type == 'maxpool':
            self.linear_pool = nn.Linear(self.dim_in, self.dim_in, bias=True)
            self.weight = nn.Linear(2 * self.dim_in, self.dim_out, bias=False)

        else:
            raise RuntimeError(f"Unknown aggregation type: {self.agg_type}")

    def forward(self, feat: torch.Tensor, edge: torch.Tensor) -> torch.Tensor:
        if self.agg_type == 'gcn':
            feat_h = feat[edge[0]]
            idx_t = edge[1]
            agg_neighbor = torch.zeros(feat.size(0), feat.size(1), dtype=torch.float32).to(feat.device)
            agg_neighbor = agg_neighbor.index_add_(0, idx_t, feat_h)
            degree = torch.bincount(idx_t, minlength=feat.size(0)).unsqueeze(1).to(feat.device)
            inv_degree = torch.where(degree == 0.0, 1.0, 1.0 / degree)
            feat_agg = agg_neighbor * inv_degree
            out = F.normalize(self.act(self.weight(feat_agg) + self.bias(feat)), 2, -1)

        elif self.agg_type == 'mean':
            feat_h = feat[edge[0]]
            idx_t = edge[1]
            agg_neighbor = torch.zeros(feat.size(0), feat.size(1), dtype=torch.float32).to(feat.device)
            agg_neighbor = agg_neighbor.index_add_(0, idx_t, feat_h)
            degree = torch.bincount(idx_t, minlength=feat.size(0)).unsqueeze(1).to(feat.device)
            inv_degree = torch.where(degree == 0.0, 1.0, 1.0 / degree)
            feat_agg = agg_neighbor * inv_degree
            out = F.normalize(self.act(self.weight(torch.cat((feat_agg, feat), 1))), 2, -1)

        elif self.agg_type == 'maxpool':
            feat = self.act(self.linear_pool(feat))
            feat_h = feat[edge[0]]
            idx_t = edge[1]
            scatter_idx = idx_t.unsqueeze(-1).repeat(1, feat.size(1))
            feat_agg = torch.zeros(feat.size(0), feat.size(1), dtype=torch.float32).to(feat.device)
            feat_agg = feat_agg.scatter_reduce(0, scatter_idx, feat_h, reduce='amax', include_self=False)
            out = F.normalize(self.act(self.weight(torch.cat((feat_agg, feat), 1))), 2, -1)

        else:
            raise RuntimeError(f"Unknown aggregation type: {self.agg_type}")

        return out

class GraphSage(nn.Module):
    def __init__(self, num_layers: int, dim_in: int, dim_hidden: int, dim_out: int, agg_type: str):
        super(GraphSage, self).__init__()
        self.num_layers = num_layers
        self.dim_in = dim_in
        self.dim_hidden = dim_hidden
        self.dim_out = dim_out
        self.agg_type = agg_type

        self.layers = nn.ModuleList()
        for l in range(num_layers):
            self.layers.append(GraphSageLayer(self.dim_in if l == 0 else self.dim_hidden, self.dim_hidden, agg_type))

        self.classifier = nn.Sequential(
            nn.Linear(2 * self.dim_hidden, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, feat: torch.Tensor, edge: torch.Tensor) -> torch.Tensor:
        x_in = feat
        for layer in self.layers:
            x_out = layer(x_in, edge)
            x_in = x_out
        return x_out

    def predict(self, head: torch.Tensor, tail: torch.Tensor) -> torch.Tensor:
        head_tail = torch.cat([head, tail], dim=-1)
        score = self.classifier(head_tail)
        return score

# GAT
# GAT
class GATLayer(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, dropout: float = 0.5, alpha: float = 0.2) -> None:
        super(GATLayer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.dropout = dropout
        self.alpha = alpha

        self.W = nn.Parameter(torch.empty(size=(in_dim, out_dim)))
        nn.init.xavier_uniform_(self.W.data)
        self.a = nn.Parameter(torch.empty(size=(2 * out_dim, 1)))
        nn.init.xavier_uniform_(self.a.data)

        self.leakyrelu = nn.LeakyReLU(negative_slope=self.alpha)
        self.batch_norm = nn.BatchNorm1d(out_dim)
        self.dropout_layer = nn.Dropout(p=self.dropout)
        self.residual = nn.Linear(in_dim, out_dim)

    def forward(self, feat: torch.Tensor, edges: torch.Tensor) -> torch.Tensor:
        message = feat @ self.W
        attn_src = message @ self.a[:self.out_dim, :]
        attn_dst = message @ self.a[self.out_dim:, :]

        src, dst = edges
        attn_scores = self.leakyrelu(attn_src[src] + attn_dst[dst])
        attn_scores = attn_scores - attn_scores.max()  # for stabilization of softmax

        # Edge softmax
        exp_attn_scores = attn_scores.exp()
        exp_sum = torch.zeros((feat.shape[0], 1), device=feat.device).scatter_add_(
            dim=0,
            index=dst.unsqueeze(1),
            src=exp_attn_scores
        ) + 1e-10  # Prevent division by zero

        attn_coeffs = exp_attn_scores / exp_sum[dst]
        attn_coeffs = self.dropout_layer(attn_coeffs)

        # Weighted aggregation
        out = torch.zeros_like(message, device=feat.device).scatter_add_(
            dim=0,
            index=dst.unsqueeze(1).expand(-1, self.out_dim),
            src=message[src] * attn_coeffs
        )
        out += self.residual(feat)  # Residual connection
        out = self.batch_norm(out)
        return out

#GAT
class GAT(nn.Module):
    def __init__(self, dim_in: int, dim_hidden: int, dim_out: int, dropout: float = 0.5, alpha: float = 0.2, num_heads: int = 8) -> None:
        super(GAT, self).__init__()
        self.dim_in = dim_in
        self.dim_hidden = dim_hidden
        self.dim_out = dim_out
        self.dropout = dropout
        self.num_heads = num_heads

        self.attn_heads1 = nn.ModuleList()
        for _ in range(num_heads):
            self.attn_heads1.append(
                GATLayer(self.dim_in, self.dim_hidden, dropout=dropout, alpha=alpha)
            )

        self.attn_heads2 = nn.ModuleList()
        for _ in range(num_heads):
            self.attn_heads2.append(
                GATLayer(self.dim_hidden * num_heads, self.dim_hidden, dropout=dropout, alpha=alpha)
            )

        self.output_layer = GATLayer(self.dim_hidden * num_heads, self.dim_out, dropout=dropout, alpha=alpha)
        self.residual = nn.Linear(self.dim_in, self.dim_out)

        self.classifier = nn.Sequential(
            nn.Linear(2 * self.dim_out, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, feat: torch.Tensor, edge: torch.Tensor) -> torch.Tensor:
        x_in = feat

        multi_head_out1 = []
        for attn_head in self.attn_heads1:
            multi_head_out1.append(attn_head(x_in, edge))
        x_out1 = torch.cat(multi_head_out1, dim=-1)

        multi_head_out2 = []
        for attn_head in self.attn_heads2:
            multi_head_out2.append(attn_head(x_out1, edge))
        x_out2 = torch.cat(multi_head_out2, dim=-1)

        x_out = self.output_layer(x_out2, edge)
        x_out += self.residual(x_in)  # Residual connection
        return x_out

    def predict(self, head: torch.Tensor, tail: torch.Tensor) -> torch.Tensor:
        head_tail = torch.cat([head, tail], dim=-1)
        score = self.classifier(head_tail)
        return score

# Define Train, Test, Predict Frameworks

In [9]:
def train(model, optimizer, train_loader, device):
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)

        pos_edge_index = data.edge_index
        neg_edge_index = data.neg_edge_index

        pos_head = out[pos_edge_index[0]]
        pos_tail = out[pos_edge_index[1]]
        neg_head = out[neg_edge_index[0]]
        neg_tail = out[neg_edge_index[1]]

        pos_pred = model.predict(pos_head, pos_tail)
        neg_pred = model.predict(neg_head, neg_tail)

        pos_loss = F.binary_cross_entropy_with_logits(pos_pred, torch.ones_like(pos_pred))
        neg_loss = F.binary_cross_entropy_with_logits(neg_pred, torch.zeros_like(neg_pred))
        loss = pos_loss + neg_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)

def test(model, loader, device):
    model.eval()
    auc = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index)

            pos_edge_index = data.edge_index
            neg_edge_index = data.neg_edge_index

            pos_head = out[pos_edge_index[0]]
            pos_tail = out[pos_edge_index[1]]
            neg_head = out[neg_edge_index[0]]
            neg_tail = out[neg_edge_index[1]]

            pos_pred = model.predict(pos_head, pos_tail)
            neg_pred = model.predict(neg_head, neg_tail)
            preds = torch.cat([pos_pred, neg_pred])

            pos_labels = torch.ones_like(pos_pred)
            neg_labels = torch.zeros_like(neg_pred)
            labels = torch.cat([pos_labels, neg_labels])

            auc += roc_auc_score(labels.cpu(), preds.cpu())

    return auc / len(loader)

def predict(model, head, tail, data, device):
    model.eval()
    with torch.no_grad():
        data = data.to(device)
        out = model(data.x, data.edge_index)
        head_feature = out[head]
        tail_feature = out[tail]
        score = model.predict(head_feature, tail_feature)
        reverse_score = model.predict(tail_feature, head_feature)
        return torch.sigmoid(score).item(), torch.sigmoid(reverse_score).item()

# Train

In [10]:
# Choose model from ["GCN", "GraphSage_mean", "GraphSage_maxpool", "GAT"]
def train_model(epoch, model_type, data_train, data_test, train_loader, test_loader, device):
    EPOCH = epoch
    MODEL = model_type

    if MODEL == "GCN":
        model = GraphSage(num_layers=2, dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, agg_type="gcn")
    elif MODEL == "GraphSage_mean":
        model = GraphSage(num_layers=2, dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, agg_type="mean")
    elif MODEL == "GraphSage_maxpool":
        model = GraphSage(num_layers=2, dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, agg_type="maxpool")
    elif MODEL == "GAT":
        model = GAT(dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, dropout = 0.5, alpha = 0.2, num_heads = 8)

    model = model.to(device)

    data_train = data_train.to(device)
    data_test = data_test.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4)

    patience = 3 # for early stopping
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(EPOCH):
        loss = train(model, optimizer, train_loader, device)
        if epoch % 10 == 0:
            test_auc = test(model, test_loader, device)
            train_auc = test(model, train_loader, device)
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train AUC: {train_auc:.4f}, Test AUC: {test_auc:.4f}')
            if loss < best_loss:
                best_loss = loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve == patience:
                    print("Early stopping triggered")
                    break
    return model

# calculate prediction accuracy
def test_model(model, data_total, data_test, device):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    head_list, tail_list = data_test.edge_index
    head_list = list(head_list)
    tail_list = list(tail_list)

    pos = 0
    for idx in range(len(head_list)):
        head = head_list[idx]
        tail = tail_list[idx]
        prediction, reverse_prediction = predict(model, head, tail, data_total, device)
        if prediction > reverse_prediction:
            pos += 1

    print(f"Prediction accuracy: {pos/len(head_list)}")

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# select model from model_types
model_types = ["GCN", "GraphSage_mean", "GraphSage_maxpool", "GAT"]
epoch = 600

In [12]:
gcn_model = train_model(epoch, "GCN", data_train, data_test, train_loader, test_loader, device)
test_model(gcn_model, data_total, data_test, device)

Epoch: 000, Loss: 1.3873, Train AUC: 0.6845, Test AUC: 0.6416
Epoch: 010, Loss: 1.0512, Train AUC: 0.8673, Test AUC: 0.8499
Epoch: 020, Loss: 0.7299, Train AUC: 0.9224, Test AUC: 0.8947
Epoch: 030, Loss: 0.6726, Train AUC: 0.9324, Test AUC: 0.9151
Epoch: 040, Loss: 0.6301, Train AUC: 0.9396, Test AUC: 0.9265
Epoch: 050, Loss: 0.5747, Train AUC: 0.9525, Test AUC: 0.9396
Epoch: 060, Loss: 0.5589, Train AUC: 0.9616, Test AUC: 0.9470
Epoch: 070, Loss: 0.4913, Train AUC: 0.9649, Test AUC: 0.9502
Epoch: 080, Loss: 0.4574, Train AUC: 0.9694, Test AUC: 0.9543
Epoch: 090, Loss: 0.4691, Train AUC: 0.9711, Test AUC: 0.9531
Epoch: 100, Loss: 0.4094, Train AUC: 0.9746, Test AUC: 0.9579
Epoch: 110, Loss: 0.3893, Train AUC: 0.9780, Test AUC: 0.9564
Epoch: 120, Loss: 0.3752, Train AUC: 0.9795, Test AUC: 0.9548
Epoch: 130, Loss: 0.3621, Train AUC: 0.9809, Test AUC: 0.9555
Epoch: 140, Loss: 0.3495, Train AUC: 0.9816, Test AUC: 0.9583
Epoch: 150, Loss: 0.3554, Train AUC: 0.9821, Test AUC: 0.9555
Epoch: 1

In [13]:
graphsage_mean_model = train_model(epoch, "GraphSage_mean", data_train, data_test, train_loader, test_loader, device)
test_model(graphsage_mean_model, data_total, data_test, device)

Epoch: 000, Loss: 1.3866, Train AUC: 0.6542, Test AUC: 0.6416
Epoch: 010, Loss: 0.9678, Train AUC: 0.8702, Test AUC: 0.8476
Epoch: 020, Loss: 0.7283, Train AUC: 0.9228, Test AUC: 0.8979
Epoch: 030, Loss: 0.6678, Train AUC: 0.9322, Test AUC: 0.9151
Epoch: 040, Loss: 0.6233, Train AUC: 0.9410, Test AUC: 0.9292
Epoch: 050, Loss: 0.5661, Train AUC: 0.9474, Test AUC: 0.9387
Epoch: 060, Loss: 0.5470, Train AUC: 0.9590, Test AUC: 0.9477
Epoch: 070, Loss: 0.4963, Train AUC: 0.9646, Test AUC: 0.9534
Epoch: 080, Loss: 0.4602, Train AUC: 0.9694, Test AUC: 0.9571
Epoch: 090, Loss: 0.4396, Train AUC: 0.9732, Test AUC: 0.9591
Epoch: 100, Loss: 0.4024, Train AUC: 0.9759, Test AUC: 0.9623
Epoch: 110, Loss: 0.3923, Train AUC: 0.9781, Test AUC: 0.9616
Epoch: 120, Loss: 0.3733, Train AUC: 0.9797, Test AUC: 0.9611
Epoch: 130, Loss: 0.3573, Train AUC: 0.9809, Test AUC: 0.9621
Epoch: 140, Loss: 0.3503, Train AUC: 0.9819, Test AUC: 0.9635
Epoch: 150, Loss: 0.3505, Train AUC: 0.9824, Test AUC: 0.9618
Epoch: 1

In [14]:
graphsage_max_model = train_model(epoch, "GraphSage_maxpool", data_train, data_test, train_loader, test_loader, device)
test_model(graphsage_max_model, data_total, data_test, device)

Epoch: 000, Loss: 1.3872, Train AUC: 0.7294, Test AUC: 0.7116
Epoch: 010, Loss: 1.2419, Train AUC: 0.8562, Test AUC: 0.8461
Epoch: 020, Loss: 0.8012, Train AUC: 0.9103, Test AUC: 0.9023
Epoch: 030, Loss: 0.7140, Train AUC: 0.9263, Test AUC: 0.9150
Epoch: 040, Loss: 0.6773, Train AUC: 0.9328, Test AUC: 0.9246
Epoch: 050, Loss: 0.6501, Train AUC: 0.9365, Test AUC: 0.9273
Epoch: 060, Loss: 0.6074, Train AUC: 0.9455, Test AUC: 0.9332
Epoch: 070, Loss: 0.6041, Train AUC: 0.9440, Test AUC: 0.9337
Epoch: 080, Loss: 0.5555, Train AUC: 0.9553, Test AUC: 0.9416
Epoch: 090, Loss: 0.5077, Train AUC: 0.9626, Test AUC: 0.9473
Epoch: 100, Loss: 0.4819, Train AUC: 0.9680, Test AUC: 0.9526
Epoch: 110, Loss: 0.4734, Train AUC: 0.9662, Test AUC: 0.9506
Epoch: 120, Loss: 0.4364, Train AUC: 0.9718, Test AUC: 0.9563
Epoch: 130, Loss: 0.4052, Train AUC: 0.9752, Test AUC: 0.9581
Epoch: 140, Loss: 0.4070, Train AUC: 0.9768, Test AUC: 0.9603
Epoch: 150, Loss: 0.3772, Train AUC: 0.9789, Test AUC: 0.9616
Epoch: 1

In [15]:
gat_model = train_model(epoch, "GAT", data_train, data_test, train_loader, test_loader, device)
test_model(gat_model, data_total, data_test, device)

Epoch: 000, Loss: 1.4095, Train AUC: 0.6933, Test AUC: 0.6151
Epoch: 010, Loss: 0.8029, Train AUC: 0.9078, Test AUC: 0.8989
Epoch: 020, Loss: 0.7135, Train AUC: 0.9249, Test AUC: 0.9158
Epoch: 030, Loss: 0.6717, Train AUC: 0.9312, Test AUC: 0.9262
Epoch: 040, Loss: 0.6228, Train AUC: 0.9384, Test AUC: 0.9352
Epoch: 050, Loss: 0.5007, Train AUC: 0.9542, Test AUC: 0.9496
Epoch: 060, Loss: 0.4246, Train AUC: 0.9709, Test AUC: 0.9653
Epoch: 070, Loss: 0.3886, Train AUC: 0.9765, Test AUC: 0.9721
Epoch: 080, Loss: 0.3721, Train AUC: 0.9782, Test AUC: 0.9757
Epoch: 090, Loss: 0.3610, Train AUC: 0.9795, Test AUC: 0.9777
Epoch: 100, Loss: 0.3533, Train AUC: 0.9805, Test AUC: 0.9785
Epoch: 110, Loss: 0.3580, Train AUC: 0.9809, Test AUC: 0.9786
Epoch: 120, Loss: 0.3616, Train AUC: 0.9815, Test AUC: 0.9787
Epoch: 130, Loss: 0.3466, Train AUC: 0.9813, Test AUC: 0.9786
Epoch: 140, Loss: 0.3378, Train AUC: 0.9823, Test AUC: 0.9783
Epoch: 150, Loss: 0.3359, Train AUC: 0.9828, Test AUC: 0.9795
Epoch: 1

# Application: 6 vs 6 win strategy (ordering)


Implementing an application that dynamically suggests the Pokemon you should send out to win when the order of the opponent's Pokemon appears randomly.

In [16]:
def node_feature_update(winner, loser, data_battle):
  """
  Update the node feature of the winner and loser pokemon after the battle

  INPUT
  winner: winner pokemon id
  loser: loser pokemon id
  data_battle: data object for the battle

  """
  damage = sum(data_battle.x[loser][i].item() for i in range(2, 8))
  data_battle.x[winner][1] -= damage
  data_battle.x[winner][0] = sum(data_battle.x[winner][i].item() for i in range(1, 8))

  return data_battle


def find_pokemon(model, opponents, opp_idx, remained_ours, data_battle, my_pokemon=None):
  """
  If my_pokemon is None, find the pokemon to fight against the opponent pokemon and return the result.
  If there is no pokemon that can win, choose the pokemon with the lowest stats among them and update the node feature
  If there are pokemons that can win, choose the pokemon with the lowest probability of winning among them and update the node feature

  If my_pokemon is not None, return the result of the fight.

  INPUT
  model: gcn model, graphsage_mean model, graphsage_max_model, gat_model
  opponents: list of opponent pokemon id
  ours: list of our alive pokemon id
  opp_idx: index of the opponent pokemon in the opponents list
  data_battle: data object for the battle
  my_pokemon: my pokemon that won in previous round. None if we lose.

  OUTPUT
  win: 1 if we lose, 2 if we win (0: default)
  my_pokemon: my pokemon participating in this round / if my_pokemon is not None, then return my_pokemon is None
  """
  win = 0
  node1 = opponents[opp_idx]
  my_winner_pokemon = [] # (pokemon_id, win_prob) list

  # 첫 라운드거나 이전 라운드에서 우리가 패배한 경우 (새로운 my_pokemon을 찾아야 하는 경우)
  if my_pokemon==None:
    # 남아있는 내 포켓몬들을 다 돌아보면서 승리할 확률이 더 높은 pokemon들을 my_winner_pokemon에 prediction score와 함께 append
    for our_idx in range(len(remained_ours)):
      node2 = remained_ours[our_idx]
      prediction, reverse_prediction = predict(model, node1, node2, data_battle.to(device), device) # 우리 pokemon이 이길 prediction score
      if prediction > reverse_prediction:
        my_winner_pokemon.append((remained_ours[our_idx], prediction))

    if not len(my_winner_pokemon):
      # 이길 수 있는 pokemon이 없다면 전체적인 stat이 높은 순서로 정렬하여 가장 약한 pokemon이 나가도록 한다.
      sorted_stats_idx = sorted(range(len(remained_ours)), key=lambda i: data_battle.x[remained_ours[i]][0].item())

      my_pokemon = remained_ours[sorted_stats_idx[0]] # sorting한 것에서 맨 앞에(stat이 가장 작은) 있는 index의 pokemon을 내보낸다.
      win = 1 # we lose
    else:
      # 이길 수 있는 pokemon이 있다면 -> prob 크기 순서대로 정렬하고 prob 가장 작은 pokemon을 ordering에 추가
      my_winner_pokemon = sorted(my_winner_pokemon, key=lambda pokemon_pair: pokemon_pair[1]) # prediction score 값을 토대로 sorting
      winner = my_winner_pokemon[0][0]
      my_pokemon = winner
      win = 2 # we win

    return win, my_pokemon

  # 이전 라운드에서 우리 포켓몬이 이긴 경우
  else:
    # opponents[opp_idx]와 my_pokemon의 승부 결과를 return해야 한다.
    node2 = my_pokemon
    prediction, reverse_prediction = predict(model, node1, node2, data_battle.to(device), device) # 우리 pokemon이 이길 prediction score
    if prediction > reverse_prediction:
      win = 2 # we win
    else:
      win = 1
    return win, None

def search_pokemon_name(pokemon_id):
  for i in range(len(pokemon_data)):
    if pokemon_data.loc[i]["#"]==pokemon_id:
      return pokemon_data.loc[i]["Name"]

def print_simulate_msg(win, opponent, our):
  opponent_pokemon_name = search_pokemon_name(opponent)
  our_pokemon_name = search_pokemon_name(our)
  print("opponent: {} ({})".format(opponent, opponent_pokemon_name))
  print("our: {} ({})".format(our, our_pokemon_name))
  if (win ==1):
    # opponent win
    print("winner: {}".format(opponent))
  else:
    # our win
    print("winner: {}".format(our))
  print("")

def update_remained_pokemon(win, my_pokemon, opp_idx, opponent_pokemon, remained_opponents, remained_ours, data_battle):
  """
  Based on the result of the fight, update the opponent/our remained pokemon list and update the survived pokemon hp feature value.

  INPUT
  win: the result of the fight
  my_pokemon: the pokemon fought in this round

  """
  # 패배한 포켓몬은 remained list에서 제거
  if (win == 1):
    # opponent win -> our pokemon이 제거되어야 한다
    data_battle = node_feature_update(opponents[opp_idx], my_pokemon, data_battle)

    remained_ours.remove(my_pokemon)
  else:
    # we win -> opponent pokemon이 제거되어야 한다.
    data_battle = node_feature_update(my_pokemon, opponents[opp_idx], data_battle)

    remained_opponents.remove(opponents[opp_idx])
    opp_idx += 1 # battle 할 다음 opponent 포켓몬을 indexing 하도록 한다.

  return remained_opponents, remained_ours, opp_idx, data_battle

In [17]:
# model input: data, edge_index, neg_edge_index
# edge index를 생성하여 predict를 하자.
# node 1(head): loser, node 2(tail): winner이라는 것에 대해서 추측하는 것

# 6마리의 pokemon을 받는다.
# edge_index: [[losers], [winners]]인데 그냥 [[opponent's pokemons], [our pokemons]]로 넣고 edge prediction 진행
def simulate(model, opponents, ours, data_battle):
  """
  Simulate the battle between our 6 pokemons and opponent's 6 pokemons

  INPUT
  model: gcn model, graphsage_mean model, graphsage_max_model, gat_model
  opponents: list of opponent's 6 pokemon ids
  ours: list of our 6 pokemon ids
  data_battle: data object for the battle
  """
  random.shuffle(opponents) # opponent
  win = 0 # 0: start, 1: opponent win , 2: ours win
  opp_idx, our_idx = 0, 0
  remained_opponents, remained_ours = copy.deepcopy(opponents), ours
  round = 1 # round #

  while (len(remained_opponents)!=0 and len(remained_ours)!=0): # ours 또는 opponents 중 하나가 0이 될 때까지 repeat
    print("Round {}.".format(round))
    if win==2: # 전 라운드에 우리가 이겼다면 my_pokemon이 input으로 들어가야 한다.
      win, _ = find_pokemon(model, opponents, opp_idx, remained_ours, data_battle, my_pokemon)
    else: # 상대 팀이 이겼었다면 my_pokemon 새로 뽑아야 한다.
      win, my_pokemon = find_pokemon(model, opponents, opp_idx, remained_ours, data_battle)
    print_simulate_msg(win, opponents[opp_idx], my_pokemon)
    remained_opponents, remained_ours, opp_idx,data_battle = update_remained_pokemon(win, my_pokemon, opp_idx, opponents[opp_idx], remained_opponents, remained_ours, data_battle)

    round += 1

  if (len(remained_ours)==0):
    print("You lose...")

  else:
    print("You win!")


In [18]:

opponents = [132, 155, 610, 382, 100, 519]
ours = [718, 357, 775, 356, 123, 635]


model = [gcn_model, graphsage_mean_model, graphsage_max_model, gat_model]
data_battle = copy.deepcopy(data_total)
simulate(model[3], opponents, ours, data_battle)

Round 1.
opponent: 382 (Milotic)
our: 123 (Kangaskhan)
winner: 123

Round 2.
opponent: 155 (Snorlax)
our: 123 (Kangaskhan)
winner: 155

Round 3.
opponent: 155 (Snorlax)
our: 635 (Gothita)
winner: 635

Round 4.
opponent: 610 (Basculin)
our: 635 (Gothita)
winner: 610

Round 5.
opponent: 610 (Basculin)
our: 356 (Spoink)
winner: 610

Round 6.
opponent: 610 (Basculin)
our: 718 (Chespin)
winner: 610

Round 7.
opponent: 610 (Basculin)
our: 357 (Grumpig)
winner: 610

Round 8.
opponent: 610 (Basculin)
our: 775 (Sliggoo)
winner: 610

You lose...


## Short output simulation

In [19]:
# # model input: data, edge_index, neg_edge_index
# # edge index를 생성하여 predict를 하자.
# # node 1(head): loser, node 2(tail): winner이라는 것에 대해서 추측하는 것

# # 6마리의 pokemon을 받는다.
# # edge_index: [[losers], [winners]]인데 그냥 [[opponent's pokemons], [our pokemons]]로 넣고 edge prediction 진행
# def short_simulate(model, opponents, ours, data_battle):
#   """
#   Simulate the battle between our 6 pokemons and opponent's 6 pokemons (short result version)

#   INPUT
#   model: gcn model, graphsage_mean model, graphsage_max_model, gat_model
#   opponents: list of opponent's 6 pokemon ids
#   ours: list of our 6 pokemon ids
#   data_battle: data object for the battle
#   """
#   random.shuffle(opponents) # opponent
#   win = 0 # 0: start, 1: opponent win , 2: ours win
#   opp_idx, our_idx = 0, 0
#   remained_opponents, remained_ours = copy.deepcopy(opponents), ours
#   round = 1 # round #
#   printing = True

#   while (len(remained_opponents)!=0 and len(remained_ours)!=0): # ours 또는 opponents 중 하나가 0이 될 때까지 repeat
#     if (len(remained_opponents)==1 or len(remained_ours)==1):
#       printing=True
#     if printing:
#       print("Round {}.".format(round))
#     if win==2: # 전 라운드에 우리가 이겼다면 my_pokemon이 input으로 들어가야 한다.
#       win, _ = find_pokemon(model, opponents, opp_idx, remained_ours, data_battle, my_pokemon)
#     else: # 상대 팀이 이겼었다면 my_pokemon 새로 뽑아야 한다.
#       win, my_pokemon = find_pokemon(model, opponents, opp_idx, remained_ours, data_battle)

#     if printing:
#       print_simulate_msg(win, opponents[opp_idx], my_pokemon)
#     if round == 2:
#       print("...")
#       print()
#       printing=False

#     remained_opponents, remained_ours, opp_idx,data_battle = update_remained_pokemon(win, my_pokemon, opp_idx, opponents[opp_idx], remained_opponents, remained_ours, data_battle)

#     round += 1

#   if (len(remained_ours)==0):
#     print("You lose...")

#   else:
#     print("You win!")

In [20]:
# ours = [132, 155, 610, 382, 100, 519]
# opponents = [718, 357, 775, 356, 123, 635]


# model = [gcn_model, graphsage_mean_model, graphsage_max_model, gat_model]
# data_battle = copy.deepcopy(data_total)
# short_simulate(model[3], opponents, ours, data_battle)